In [ ]:
import pyspark
import json

In [ ]:
conf = pyspark.SparkConf()
conf.setAppName("Twitter page rank")
conf.setMaster("local[*]")

In [ ]:
sc = pyspark.SparkContext(conf = conf)

In [ ]:
twitterLines = sc.textFile("twitter_closed.jsonl")

In [ ]:
followers = twitterLines.map(lambda line: json.loads(line))

In [ ]:
links = followers\
    .keyBy(lambda d: d["user_id"])\
    .mapValues(lambda d: d["following_id"])

In [ ]:
N = links.count()
d = 0.85

In [ ]:
ranks = links\
    .keyBy(lambda usuario: usuario[0])\
    .map(lambda usuario: (usuario[0], 1 / N))

In [ ]:
iterations = 10

In [ ]:
def calcula_contribuciones(registro):
    """
    Calcula las contribuciones de un `usuario` hacia las personas a las que sigue
    
    args:
        registro: una tupla de tuplas, de la siguiente forma: (usuario, (seguidos, page_rank))
    
    returns:
        una lista de tuplas de la forma (usuario_seguido, contribución pr)
    """
    
    resultados = list()
    (usuario, (seguidos, page_rank_usuario)) = registro

    for seguido in seguidos:
        resultados.append((seguido, page_rank_usuario / len(seguidos)))
    resultados.append((usuario, 0))
    
    return resultados

In [ ]:
for i in range(iterations):
    
    contributions = links.join(ranks)\
        .flatMap(calcula_contribuciones)
    
    ranks = contributions\
        .reduceByKey(lambda a, b: a + b)\
        .mapValues(lambda v: (1 - d) / N + d * v)

In [ ]:
results = dict(ranks.collect())

In [ ]:
results[2582486138]

In [ ]:
results[1306760288243462147]

In [ ]:
sc.stop()